# Getting all available Swiss supermarkets using the overpass Web API

## Overpass turbo query to get all available supermarkets in Switzerland

In [ ]:
from pandas import json_normalize
import requests
import json
import folium

# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["shop"="supermarket"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('supermarkets.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)
df.head(5)

## Plot supermarkets on map

In [ ]:
# Subset of supermarkets by brand
locations = df[["lat", "lon", "tags.brand", "tags.shop"]].loc[df["tags.brand"].isin(['Coop', 'Migros'])]
print(locations.head(5))

# Subsets
locations_coop = df[["lat", "lon", "tags.brand",
                     "tags.shop"]].loc[df["tags.brand"].isin(['Coop'])]
locations_migros = df[["lat", "lon", "tags.brand",
                       "tags.shop"]].loc[df["tags.brand"].isin(['Migros'])]

# Marker symbols
url_01 = 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-green.png'
url_02 = 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-orange.png'

# Create map
map = folium.Map(location=[locations.lat.mean(),
                           locations.lon.mean()],
                 zoom_start=8,
                 control_scale=True)

# Add lat/lon of Coop supermarkets
for i in range(0, len(locations_coop)):
    folium.Marker(location=(locations_coop.iloc[i]['lat'],
                            locations_coop.iloc[i]['lon']),
                  popup=locations_coop.iloc[i]['tags.brand'],
                  icon=folium.features.CustomIcon(url_01, icon_size=(16, 28))).add_to(map)
    
# Add lat/lon of Migros supermarkets
for i in range(0, len(locations_migros)):
    folium.Marker(location=(locations_migros.iloc[i]['lat'],
                            locations_migros.iloc[i]['lon']),
                  popup=locations_migros.iloc[i]['tags.brand'],
                  icon=folium.features.CustomIcon(url_02, icon_size=(16, 28))).add_to(map)

# Plot map
map

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')